### Aufgabe 2:
- Ja lässt sich detaillierter Ausarbeiten.
- Kommilitonen haben die einzelnen Schritte bereits als Mapper und Reducer Funktionen definiert.
- Das kann man hinzufügen.

In [1]:
import tempfile
import numpy as np
from itertools import combinations

from util import rent_in_rome, enron_email
from MinHash import MinHash, KShingles
from LSH import LSH, MRLSH

### Aufgabe 3:
- Implementiere MinHash ohne MapReduce
- Evaluiere auf dem housing-dataset
- Sind die MinHash-ähnlichen Instanzen auch Jaccard-ähnlich?
- Was liefert die manuelle Inspektion der Strings? Sind sie ähnlich?

In [2]:
data = rent_in_rome()

data.head()

,Title,Short Description
0,Studio accessoriato vicino metro A Furio Camillo,Affitto studio a professionisti preferibilment...
1,"Negozio 169Mq per laboratorio, ufficio, studio...","Privato affitta negozio 169 mq, al piano terra..."
2,Negozio in tiburtina centro,Negozio c/1 roma tiburtina centro via eugenio ...
3,Studio medico via anapo parco nemorense,"Studio medico avviato, composto da tre studi c..."
4,Cerco: Appartamento per donna lavoratrice refe...,"Donna lavoratrice, non residente, con reddito ..."


In [3]:
minhash = MinHash(k=4)
data['MinHash Signature'] = minhash(data['Short Description'])

data.head()

,Title,Short Description,MinHash Signature
0,Studio accessoriato vicino metro A Furio Camillo,Affitto studio a professionisti preferibilment...,"[0.0, 43.0, 9.0, 13.0, 71.0, 21.0, 28.0, 147.0..."
1,"Negozio 169Mq per laboratorio, ufficio, studio...","Privato affitta negozio 169 mq, al piano terra...","[19.0, 22.0, 35.0, 11.0, 9.0, 6.0, 11.0, 13.0,..."
2,Negozio in tiburtina centro,Negozio c/1 roma tiburtina centro via eugenio ...,"[483.0, 0.0, 21.0, 218.0, 314.0, 330.0, 11.0, ..."
3,Studio medico via anapo parco nemorense,"Studio medico avviato, composto da tre studi c...","[483.0, 74.0, 104.0, 406.0, 307.0, 13.0, 11.0,..."
4,Cerco: Appartamento per donna lavoratrice refe...,"Donna lavoratrice, non residente, con reddito ...","[19.0, 82.0, 9.0, 45.0, 18.0, 6.0, 57.0, 25.0,..."


In [4]:
jaccard = lambda a, b: len(a.intersection(b)) / len(a.union(b))
accuracy = lambda a, b: np.mean(a == b)

k_shingles = KShingles(4)

data['Document'] = data['Short Description'].apply(k_shingles).apply(set)

In [5]:
high_minhash_similarity = [jaccard(set1, set2)
                           for (i, (set1, sig1)), (j, (set2, sig2))
                           in combinations(data[['Document', 'MinHash Signature']].iterrows(), 2)
                           if accuracy(sig1, sig2) > 0.9]

In [6]:
print("Min Jaccard-Similarity of High-MinHash Similarity values", np.min(high_minhash_similarity))
print("Mean Jaccard-Similarity of High-MinHash Similarity values", np.mean(high_minhash_similarity))

Min Jaccard-Similarity of High-MinHash Similarity values 0.8546255506607929
Mean Jaccard-Similarity of High-MinHash Similarity values 0.9979909916082947


### Aufgabe 4:
- Implementierung von LSH anhand MinHash Implementierung.
- Evaluierung an Enron-Email-Dataset.

Ohne MRJob

In [7]:
minhash = MinHash(k=4)
lsh = LSH(r=5)

In [8]:
minhash_signatures = minhash(data['Short Description'])

In [9]:
buckets, candidate_pairs = lsh(minhash_signatures)

In [10]:
print("Candidate Pairs:", *candidate_pairs, sep='\n')

Candidate Pairs:
(1271, 2527)
(2272, 2378)
(484, 1659)
(735, 1241)
(485, 1066)
(153, 327)
(2307, 2335)
(221, 2188)
(2219, 2222)
(956, 2408)
(317, 2421)
(1000, 1660)
(1396, 1660)
(418, 2473)
(363, 2308)
(1197, 1945)
(2193, 2435)
(1329, 1835)
(1007, 1821)
(904, 905)
(353, 802)
(1330, 2474)
(418, 1197)
(342, 384)
(1801, 1830)
(2439, 2612)
(154, 1087)
(460, 2591)
(259, 2601)
(2196, 2625)
(1924, 2166)
(155, 1726)
(64, 2096)
(83, 160)
(286, 977)
(74, 2555)
(625, 2429)
(2130, 2271)
(697, 1780)
(1987, 2222)
(22, 2099)
(757, 2319)
(2061, 2171)
(2253, 2341)
(1131, 1483)
(1263, 1373)
(2610, 2615)
(117, 158)
(2247, 2563)
(260, 466)
(868, 2012)
(2219, 2464)
(2045, 2577)
(64, 1988)
(1462, 1704)
(2042, 2050)
(992, 1260)
(221, 1154)
(2325, 2338)
(269, 636)
(353, 1044)
(1791, 1879)
(418, 1439)
(249, 2384)
(1, 67)
(1924, 2408)
(155, 1968)
(890, 2188)
(64, 2338)
(2097, 2336)
(1330, 1440)
(1987, 2464)
(757, 2561)
(2253, 2583)
(419, 1748)
(1219, 2363)
(1066, 1330)
(1654, 1766)
(1263, 1615)
(287, 582)
(2222

In [11]:
data.iloc[354]['Short Description'], data.iloc[381]['Short Description']

("Prati Via di Cola di Rienzo all''interno di uno stabile di rappresentanza si affittano uffici di 150 mq. Sono disponibili anche tagli da 350 mq e 650 mq (intero piano) oppure, si possono affittare piÃ¹ piani per un totale di 3.200 mq. Tel. 06.36006910 ",
 "Prati Via di Cola di Rienzo all''interno di uno stabile di rappresentanza si affittano uffici di 150 mq. Sono disponibili anche tagli da 350 mq e 650 mq (intero piano) oppure, si possono affittare piÃ¹ piani per un totale di 3.200 mq. Tel. 06.36006910 ")

Mit MRJob

In [12]:
data = enron_email(100)


def run(input_file, output_file):
    job = MRLSH(args=[input_file])
    with job.make_runner() as runner, open(output_file, 'w') as out:
        runner.run()
        out.writelines(f"{key}\t{value}\n" for key, value in job.parse_output(runner.cat_output()))

In [13]:
minhash = MinHash(8)
data['signature'] = minhash(data['body'])

In [14]:
to_int_string = lambda x: map(str, map(int, x))

with tempfile.NamedTemporaryFile('w+') as tmp:
    signature = data['signature'].map(to_int_string).map('\t'.join)
    tmp.writelines(data['file'] + '\t' + signature + '\n')
    run(tmp.name, './candidate-pairs')

No configs specified for inline runner
